# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [71]:
#load the csv from the previous exercise
weather="output/cities.csv"
weather_df=pd.read_csv(weather)
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,kapaa,40,US,1584236519,57,22.08,-159.32,80.60,14.99
1,puerto ayora,4,EC,1584236432,64,-0.74,-90.35,82.00,5.99
2,norman wells,90,CA,1584236565,85,65.28,-126.83,23.00,2.24
3,atuona,28,PF,1584236585,67,-9.80,-139.03,82.47,8.03
4,arraial do cabo,16,BR,1584236432,88,-22.97,-42.03,78.28,12.30
...,...,...,...,...,...,...,...,...,...
570,imeni poliny osipenko,0,RU,1584236683,62,52.42,136.49,17.85,2.51
571,moshupa,0,BW,1584236683,94,-24.78,25.42,68.00,3.36
572,hukuntsi,0,BW,1584236684,53,-24.00,21.75,72.27,7.07
573,humaita,99,BR,1584236684,98,-7.51,-63.02,73.71,4.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [72]:
humidity=weather_df["Humidity"].astype(float)
locations=weather_df[["Lat","Lng"]]

In [73]:
#plot Heat meap

heat_layer=gmaps.heatmap_layer(locations,weights=humidity,
                              dissipating=False,max_intensity=5,
                              point_radius=1)






In [74]:
#show the figure

figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
#select value from a dataframe with several criteria (max temperature, wind speed, cloudiness)
weather_selected_df=weather_df.loc[(weather_df["Max Temp"]<80) & (weather_df["Max Temp"]>70) & (weather_df["Wind Speed"]<30) & (weather_df["Cloudiness"]==0)]
hotel_df=weather_selected_df.dropna(how="any")
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
118,sao miguel do araguaia,0,BR,1584236624,94,-13.28,-50.16,72.41,1.86
213,sao filipe,0,CV,1584236637,57,14.90,-24.50,73.15,13.91
282,cockburn town,0,TC,1584236645,79,21.46,-71.14,75.61,19.80
389,phalaborwa,0,ZA,1584236659,91,-23.94,31.14,70.88,5.77
390,los andes,0,CL,1584236659,38,-32.83,-70.60,73.40,6.93
398,san vicente,0,AR,1584236661,51,-26.62,-54.13,71.10,2.35
460,sabancuy,0,MX,1584236669,89,18.97,-91.18,73.90,2.75
507,ilhabela,0,BR,1584236675,86,-23.78,-45.36,77.52,12.93
527,cidreira,0,BR,1584236678,89,-30.18,-50.21,76.06,23.31
572,hukuntsi,0,BW,1584236684,53,-24.00,21.75,72.27,7.07


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
hotel_df["Hotel"]=""

In [88]:
#find hotels in radius 5000 m

#define gmaps parameter to find hotel / lodging
params={
    "key":g_key,
    "type":"lodging",
    "radius":5000
}

#get location from df
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]

    #Add keyword to params dict
    params['location']=f"{lat},{lng}"

    #use search term for hotel and radius
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and convert to json
    hotel_name=requests.get(base_url,params=params)    
    hotel_name=hotel_name.json()
    print(hotel_name)

    #populate in dataframe
    try: 
        hotel_df.loc[index,"Hotel"]=hotel_name["results"][0]["name"]

    #populate in df

    except(KeyError,IndexError):
        print("Missing result.")

{'html_attributions': [], 'results': [{'geometry': {'location': {'lat': -13.280485, 'lng': -50.17611199999999}, 'viewport': {'northeast': {'lat': -13.2791028697085, 'lng': -50.17478396970849}, 'southwest': {'lat': -13.2818008302915, 'lng': -50.17748193029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '0ad4274fd28378f4e0f40ccd5668558887890f36', 'name': 'Hotel Executivo Palace', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1080, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117199980329694174387">Edicarlos Ribeiro de sousa</a>'], 'photo_reference': 'CmRaAAAAMs89MWuWtSevXRTWOExJJkwqhpL6AlfS0NyeZbDe6I13p2CndLyg3eWxWoAD7kEIv28aeEFmltodl6uV967EzWWMxJBVKm9WFHUv5cB1h1SNMkZx6083iZjF-MFpiN-yEhCDm7jvU1hTrtw5IvNnUzH-GhSbTkLxxDw_4CVcANqCrNT7XMZwvg', 'width': 1920}], 'place_id': 'ChIJR0YnVeqaapMR9GmEaeHU4HQ', 'plus_code': {'compound_code': 'PR9F+RH São Miguel do Araguaia, State of Goiás, Brazil', 'global_code': '58RFPR9F

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAAOMt1x032sEoJLOm2IK-DPrkFZr704NpLekyP--a9tTBEytDAj3dh1c_An--MVX4dVOMsBiOTA5zF9_tS-gntHxp0VWTu-dNktbeb6iWGZOlgU9RtizdntcdEfoaPyJpu2HUY7gokBOxslfhX_O622n1OEns3GNz9ZFJserOZDQqIt16rn9d8o3u1XFLNESVQ8ddXBibEYCEcF7u6l4zT34mp42QHPipoUdF_6kkkxwRNrjqnA4csgqetVkqwLiJsqgJ-0nd8WT1J98YoUNEQvnNjJ8fL0fdInDx06Zx-yrpFO87-Xdx6xydj4Gby-71NqdLDR4JQVUOSkxtYQjVztsievYn-HyMmMPk1HDs1nKKTeZ1tCchQRQJd6R8efMvhIQRXQi-NOJ0LoKJKzOKmQJqBoUOdT6iqNaSVahrDtEwRwq_Qe3K6w', 'results': [{'geometry': {'location': {'lat': 14.8807361, 'lng': -24.4929618}, 'viewport': {'northeast': {'lat': 14.8838864, 'lng': -24.4922843197085}, 'southwest': {'lat': 14.879686, 'lng': -24.4949822802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '128be0f4b2d43b33c447adcada12752a84bdceb1', 'name': 'Tortuga B&B', 'photos': [{'height': 2268, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105528571623851186559">maria gautier</a>'], '

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAMupKyyG8CC_xNC8xX_3xbnPOTicXGsyNR2elR0fzhnAkX7SUUvORn-wVrIU_cgWwi3x45QU1XwnFMSVDq9So-9GxCLPWL5RxwvnYJLxX-PqApH76Y6-JkfRSuxye89M7hEFN_aq48m3fBQGewtH0oSMWtzox1sKUdzY6zgVyplPwkkr8J3kL9fenlidlj-_se4PRmE647innBctMqLnxXK1mKIEoQpyAAtTRGs1kdrbnGsRy14sJ9cjxfuL8WwyAK41XbMQy7IyTlJf2KJOYpVluOBTfRvEVpcHldqoyMobQXB81Q3tK9VEHPBX1NBEmp4uJyQnHjduvL0dEDNSi6jJONFFI02tbP_EjKmJmCzs3NA-e9FHNTfjOYKoukQhohIQmldAgnmi5eDH2TfJVdfY5RoUSj8lvlx2hRmIafHNC05Nkv3xsqk', 'results': [{'geometry': {'location': {'lat': 21.4595376, 'lng': -71.1474039}, 'viewport': {'northeast': {'lat': 21.4608855302915, 'lng': -71.14605036970849}, 'southwest': {'lat': 21.45818756970849, 'lng': -71.14874833029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '75da017aff790a78350d15d366fc80db93b9a38a', 'name': 'Osprey Beach Hotel', 'photos': [{'height': 1080, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110305791248348871372"

{'html_attributions': [], 'next_page_token': 'CqQCHAEAANHzjU3Fvv6CxvSpnqfpol2O8mMBnrYxvs5_XxIu5D33LTPPEHhyFPkoVWfHZwiYs6O7d97qZg0E9MAGL4x0VjS9Xd4TtraPRScXSFAfM7OEj3X9pNnVq7v0ZZXjGmrwvxeK1SLJ-EiyCjPypvQCapEzJHEEaBFuhpBnZg3E0of09yllIg-qpdmybi3MAPGtERpOfK9ihDvhywbuj2dW9GPLMweJ_nR_kRTnPZeJQjufad36juS6PxcIMv6RPTxpYDR-3pXYyTzF9b0bccg-B9vCzc454UlaaQrIL7Y3jcbNJTfgdjwHGtWxISbc-M4Q6EvzCj3jmGNUEOGeWGFGW8FPWhlxHTdFXokyOhh0yI2LriAoOahhFi4RRBdM58_-sxIQzTQe3pSCrj_LqMbscDypkRoU0caeuMG9SWpeyAvdFSo89EDW_f8', 'results': [{'geometry': {'location': {'lat': -23.957071, 'lng': 31.14432699999999}, 'viewport': {'northeast': {'lat': -23.9557456697085, 'lng': 31.1458292802915}, 'southwest': {'lat': -23.9584436302915, 'lng': 31.1431313197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '53f41ce17272ae7929add55351393b05ba4a52cb', 'name': 'Bothabelo B&B', 'opening_hours': {'open_now': False}, 'photos': [{'height': 2988, 'html_attributions': ['<a href="https://maps.google.com

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAHnIUlzkUd_aDvRTkQWJSCCrVb60Q-39X-h6C4lIuiLvLkDbUM7hlqlOx3NoXOBhPr0FvdyQ3JAY25kHNfZv2a_g61J_ZBYD-sNeYejQyey-0B5H8msIoL41S2uMBAer3bIW_7s8QGh-1tujMFgUfnAAsnZfFrNiaLur54m6z9KztzDijExN6F6ywjYE6AG7tOSWwBmz4gr_MsV02RkHTv43TFznbzLo1ZEjqbc1lC0gjTtpT1LrFWfnQTsMzQOFGPHtdJLgdMW4rmU35uOAusW5OTW_oX_NpFPDcQooOHL7MqWkM4_3k1rWCKdyPtR4z28-RU_o4GhlHDpnxAYb743Oj6q7AC4L5vK98lmvTQldRPHabyyJHMcc43xmrpfoUBIQ-I7UFkAUZgPmQRblsQygxBoUshlRHUDeW83JC4X0msCv0oslwNA', 'results': [{'geometry': {'location': {'lat': -32.82700519999999, 'lng': -70.6077042}, 'viewport': {'northeast': {'lat': -32.8256248697085, 'lng': -70.6063515197085}, 'southwest': {'lat': -32.8283228302915, 'lng': -70.6090494802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '43fee0eb1d6a34ae6af8dddc170186a5b8c98c3f', 'name': 'Hotel Los Andes', 'opening_hours': {'open_now': True}, 'photos': [{'height': 766, 'html_attributions': ['<a href="https://maps.google

{'html_attributions': [], 'results': [{'geometry': {'location': {'lat': -26.6024047, 'lng': -54.1121576}, 'viewport': {'northeast': {'lat': -26.60129940000001, 'lng': -54.11106445}, 'southwest': {'lat': -26.60572059999999, 'lng': -54.11543705}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/camping-71.png', 'id': '0121a9b2cead510e8d7017c8b0d5adfc909344a1', 'name': 'Complejo Turístico Farelluk', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3352, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111492804132366623691">Complejo Turístico Farelluk</a>'], 'photo_reference': 'CmRaAAAAN_8tIl7kopIKK3nKHot3hDSjdIXNhGUanSVkbG2WD0-HoVJbB82Kb8XM8yUnqnKMmNDFOv8FzkxEc4cFQpjH1D5s3AiRzNFm0Fi5RxmBcZecgIuVR5fTR5wZHNdQutNpEhBiF2mpydOG_OBjbhcDf-2LGhRdgF1eaimvj3iN6CR60ByDMqDBmA', 'width': 5152}], 'place_id': 'ChIJsasrecz4-ZQR5NGpG6q1n6g', 'plus_code': {'compound_code': '9VXQ+24 San Pedro, Misiones Province, Argentina', 'global_code': '58579VXQ+24'}, 'rating':

{'html_attributions': [], 'results': [{'geometry': {'location': {'lat': 18.9702945, 'lng': -91.1778951}, 'viewport': {'northeast': {'lat': 18.9715348802915, 'lng': -91.17648276970849}, 'southwest': {'lat': 18.9688369197085, 'lng': -91.1791807302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'f49f32e0e9e487ec97c7b0fd45145edcf3f17630', 'name': 'Hotel Aguilar', 'photos': [{'height': 2560, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103973154907270467898">javier camas</a>'], 'photo_reference': 'CmRaAAAAMQKF4DHtNHsQjsJLJyKuL0fgsj2jgHiY-8jlmBuMOv0S5j-T7Z04nsJ9vpmpJfnJFdITY9wS2tZC_LO8907A47_ycHJ2aWWNZOGNPhfymJOxuPNj1rMUIX-kCVvNXObrEhAnthFB_IW9lHN7Hxmh2ULPGhQZbEbU6YcvUaFpCWphDkfa3UzgTw', 'width': 1920}], 'place_id': 'ChIJnyaqsnQt94URgW-oqXbCVNI', 'plus_code': {'compound_code': 'XRCC+4R Sabancuy, Campeche, Mexico', 'global_code': '76CCXRCC+4R'}, 'rating': 3.9, 'reference': 'ChIJnyaqsnQt94URgW-oqXbCVNI', 'scope': 'GOOGLE', 'type

{'html_attributions': [], 'next_page_token': 'CqQCHAEAACgFu-FgtSwn-KD9YCoQueagnFXFXfPxafmAoBOnZ7hgUlrUnv221SdUm1jVimB1FsTBhmO1JLJ9rabwiDHHNsdQEsv0UGXvhqQNwOjwGXkVJcBp30iGxDrozUXDuinve5idmQUEXIj075KfS-DHl9dzzADs-Kyj2YwfBEndjnp9j_VKGC14XRwJg79k3wvYhbEpkeEQz55SKYsgXvMVtorIRx-qWUtgCi1RXswk9MaOPdoh7e-8qyJH5TvC6IlE8KVs7DYGf6Qjac9l2ik5YIE2Yf3Ddcwi6KxBQ8yDnx5Fp4sS_fqeRMz-IYG3N1beI5JRk-Y1k1QkWwGZsp-AmnJAkQSTFteYaAT6ZfG6M5J_OsTRTk1KrdXNvmSGdpu1UhIQgagjzK6p9MwH5M5XQN1wzRoUoMVRV467N46A_Otj2vt7R5hBB94', 'results': [{'geometry': {'location': {'lat': -30.1592308, 'lng': -50.1997331}, 'viewport': {'northeast': {'lat': -30.1578710697085, 'lng': -50.1984219697085}, 'southwest': {'lat': -30.16056903029149, 'lng': -50.2011199302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'b7df7ea63e23d1fe48b1dc74920fbdd2c37fbe59', 'name': 'Novomar Pousada e Turismo Ltda', 'photos': [{'height': 2448, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/118221283

{'html_attributions': [], 'results': [{'geometry': {'location': {'lat': -23.9993292, 'lng': 21.7913979}, 'viewport': {'northeast': {'lat': -23.9980800197085, 'lng': 21.7927389302915}, 'southwest': {'lat': -24.0007779802915, 'lng': 21.7900409697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'b282a7864667d582059e9d8f9527b992a14edc83', 'name': 'Entabeni Hotel', 'photos': [{'height': 3016, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112593927305902754153">Tshepo Molefe</a>'], 'photo_reference': 'CmRaAAAAwMoup9Si0I3huPMnoUHMmCaPxWl45qg8H_hDpsUGuV9yFXtWfyrHo7HCBv3SWzejmyTGpg5KKA_AZ-jA3pjxV5REtnXMUmW1ymEkVORVjjbXri-SYLDxep6Y8upDxJatEhC2t7wah5toscYiKJzQP6aEGhSjYFJAXLbz1gq7m-d2PUSAxxaJdQ', 'width': 4032}], 'place_id': 'ChIJ7U-ico4rAxwR90Fi3s_tMkE', 'plus_code': {'compound_code': '2Q2R+7H Hukuntsi, Botswana', 'global_code': '5G832Q2R+7H'}, 'rating': 5, 'reference': 'ChIJ7U-ico4rAxwR90Fi3s_tMkE', 'scope': 'GOOGLE', 'types': ['lod

In [89]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel
118,sao miguel do araguaia,0,BR,1584236624,94,-13.28,-50.16,72.41,1.86,Hotel Executivo Palace
213,sao filipe,0,CV,1584236637,57,14.90,-24.50,73.15,13.91,Tortuga B&B
282,cockburn town,0,TC,1584236645,79,21.46,-71.14,75.61,19.80,Osprey Beach Hotel
389,phalaborwa,0,ZA,1584236659,91,-23.94,31.14,70.88,5.77,Bothabelo B&B
390,los andes,0,CL,1584236659,38,-32.83,-70.60,73.40,6.93,Hotel Los Andes
398,san vicente,0,AR,1584236661,51,-26.62,-54.13,71.10,2.35,Complejo Turístico Farelluk
460,sabancuy,0,MX,1584236669,89,18.97,-91.18,73.90,2.75,Hotel Aguilar
507,ilhabela,0,BR,1584236675,86,-23.78,-45.36,77.52,12.93,Ilha Flat Hotel
527,cidreira,0,BR,1584236678,89,-30.18,-50.21,76.06,23.31,Novomar Pousada e Turismo Ltda
572,hukuntsi,0,BW,1584236684,53,-24.00,21.75,72.27,7.07,Entabeni Hotel


In [90]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [91]:
#add properties for figure size
figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


# Add marker layer ontop of heat map with info_box_content
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…